#### 导入库

In [1]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import os
import sys

#### 文档加载&切分&检索

In [2]:
! python ingest.py # 命令行 python3 ./DB/Process/ingest.py 制作向量数据库

python: can't open file '/home/linjw/AGI/llama2/langchain/ingest.py': [Errno 2] No such file or directory


In [3]:
### 文档查询 去向量数据库匹配相关代码
doc_query = "人工智能 突破技术 未来技术 塑造未来 革命 新一代技术 未来产业"


### 递归总结, 根据查询出来的向量数据库相关片段 循环输入进llm进行要素提取
refine_query = """
根据获取的信息，总结并提炼出人工智能的突破性技术：
1. 技术1: [描述]
2. 技术2: [描述]
3. 技术3: [描述]
4. 技术4: [描述]
5. 技术5: [描述]
6. 技术6: [描述]
7. 技术7: [描述]
8. 技术8: [描述]
9. 技术9: [描述]
10. 技术10: [描述]
"""


# 最终总结输出 总结递归的最后输出
summary_query = """在已知的信息中，确定并列出人工智能的突破性技术"""


# 回答模板 将summary_query的输出格式化为对象
response_schemas = [
    ResponseSchema(name="技术1", description="技术1的简要描述"),
    ResponseSchema(name="技术2", description="技术2的简要描述"),
    ResponseSchema(name="技术3", description="技术3的简要描述"),
    ResponseSchema(name="技术4", description="技术4的简要描述"),
    ResponseSchema(name="技术5", description="技术5的简要描述"),
    ResponseSchema(name="技术6", description="技术1的简要描述"),
    ResponseSchema(name="技术7", description="技术2的简要描述"),
    ResponseSchema(name="技术8", description="技术3的简要描述"),
    ResponseSchema(name="技术9", description="技术4的简要描述"),
    ResponseSchema(name="技术10", description="技术5的简要描述"),
]


In [4]:
sys.path.append(os.path.dirname(os.path.realpath('__file__'))+"/../")
import config.conf as cfg
from chromadb.config import Settings

# Define the folder for storing database
PERSIST_DIRECTORY = cfg.PERSIST_DIRECTORY.replace("\\langchain","")
# Define the Chroma settings
CHROMA_SETTINGS = Settings(
        persist_directory=PERSIST_DIRECTORY,
        anonymized_telemetry=False
)

embeddings_model_name = cfg.EMBEDDINGS_MODEL_NAME.replace("\\langchain","")
persist_directory = cfg.PERSIST_DIRECTORY.replace("\\langchain","")
model_path = cfg.MODEL_PATH.replace("\\langchain","")
model_n_ctx = cfg.MODEL_N_CTX
model_n_batch = cfg.MODEL_N_BATCH
target_source_chunks = 4
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
chroma_client = chromadb.PersistentClient(settings=CHROMA_SETTINGS , path=persist_directory)
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS, client=chroma_client)

ValueError: Path /home/linjw/AGI/llama2/langchain/checkpoints/paraphrase-multilingual-MiniLM-L12-v2 not found

In [ ]:
cfg.EMBEDDINGS_MODEL_NAME

'D:\\WorkSpace\\llama2\\langchain/checkpoints/paraphrase-multilingual-MiniLM-L12-v2'

##### 普通检索

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 20})

# retriever = db.as_retriever(    
#     search_type="mmr",
#     search_kwargs={'k': 20, 'lambda_mult': 0.25})


In [ ]:
docs = retriever.get_relevant_documents(doc_query)
[doc.page_content for doc in docs]

['their overall digital transformation maturity journey .\n人工智能未来\n已来，是机遇\n还是挑战\n人工智能产业近年来发展迅猛，渗透了各行各业，而新技术的革新往往\n是一把“双刃剑”，在提升企业商业利益的同时也会带来一定的风险。\n本报告对各行业领域中超过100家企业进行了调研，并根据整合的数据\n进行了深度洞察分析，其中包括人工智能未来5年的用途和影响，以及\n人工智能为企业带来的商业利益和风险。\n大中华区人工智能成熟度调研\n35',
 'their overall digital transformation maturity journey .\n人工智能未来\n已来，是机遇\n还是挑战\n人工智能产业近年来发展迅猛，渗透了各行各业，而新技术的革新往往\n是一把“双刃剑”，在提升企业商业利益的同时也会带来一定的风险。\n本报告对各行业领域中超过100家企业进行了调研，并根据整合的数据\n进行了深度洞察分析，其中包括人工智能未来5年的用途和影响，以及\n人工智能为企业带来的商业利益和风险。\n大中华区人工智能成熟度调研\n35',
 '01  <<\n新一代人工智能产业白皮书（2019 年）\n——主要应用场景研判\n新一代人工智能作为全球新一轮科技和产业变革的关键驱动力，将进一步释放科技\n革命和产业变革积蓄的巨大能量，持续探索新一代人工智能应用场景，将重构生产、分配、\n交换、消费等经济活动各环节，催生新技术、新产品、新产业、新业态、新模式，形成\n从宏观到微观各领域的智能化新需求，并创造新的经济发展的强大引擎，引发经济结构\n重大变革，深刻改变人类生产生活方式和思维模式，实现社会生产力的整体跃升。\n全球新一代人工智能产业依赖强大的技术创新积累优势，以谷歌、英特尔、微软、\n亚马逊等跨国大型科技企业为主导，充分发挥其强大的资源整合能力与持续创新功能，\n不断加快基础层底层技术研发与应用产品实践步伐，围绕智能硬件与软件核心算法产业\n上下游进行有效部署。随着技术的不断成熟和底层技术框架的开源，吸引创新企业不断\n涌入，推动产业规模持续加速增长。2018 年全球新一代人工智能产业规模超过 555.7 亿\n美元，预计 2019 年产业规模将突破 718 亿美元，带动 2022 年产业规模将超过 

In [ ]:
for doc in docs:# 来源
    print(doc.metadata)


{'author': '', 'creationDate': "D:20201022153906+08'00'", 'creator': 'Adobe InDesign 15.0 (Windows)', 'file_path': 'D:\\WorkSpace\\llama2/DB/Process/source\\2020年  大中华区人工智能成熟度调研——中国大陆，解码2020，展望数字未来-安永x微软.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20201112143019+08'00'", 'page': 34, 'producer': '99pdf.com', 'source': 'D:\\WorkSpace\\llama2/DB/Process/source\\2020年  大中华区人工智能成熟度调研——中国大陆，解码2020，展望数字未来-安永x微软.pdf', 'subject': '', 'title': '', 'total_pages': 81, 'trapped': ''}
{'author': '', 'creationDate': "D:20201022153906+08'00'", 'creator': 'Adobe InDesign 15.0 (Windows)', 'file_path': 'D:\\WorkSpace\\llama2/DB/Process/source\\2020年  大中华区人工智能成熟度调研——中国大陆，解码2020，展望数字未来-安永x微软.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20201112143019+08'00'", 'page': 34, 'producer': '99pdf.com', 'source': 'D:\\WorkSpace\\llama2/DB/Process/source\\2020年  大中华区人工智能成熟度调研——中国大陆，解码2020，展望数字未来-安永x微软.pdf', 'subject': '', 'title': '', 'total_pages': 81, 'trapped': ''}
{'author': '', 'creati

##### LLM压缩检索

In [ ]:
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import LLMChainExtractor
# from langchain.llms import OpenAI
# from langchain.retrievers.self_query.base import SelfQueryRetriever
# from langchain.chains.query_constructor.base import AttributeInfo
 
# #定义元数据的过滤条件
# metadata_field_info = [
#     AttributeInfo(
#         name="source",
#         description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
#         type="string",
#     ),
#     AttributeInfo(
#         name="page",
#         description="The page from the lecture",
#         type="integer",
#     ),
# ]
 
# #创建SelfQueryRetriever
# document_content_description = "Lecture notes"
# llm = OpenAI(temperature=0)
# retriever = SelfQueryRetriever.from_llm(
#     llm,
#     db,
#     document_content_description,
#     metadata_field_info,
#     verbose=True
# )
# def pretty_print_docs(docs):
#     print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))
    
# # Wrap our vectorstore
# llm = OpenAI(temperature=0)
# compressor = LLMChainExtractor.from_llm(llm)
 
# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=compressor,
#     base_retriever=db.as_retriever()
# )

# question = "what did they say about matlab?"
# compressed_docs = compression_retriever.get_relevant_documents(question)
# pretty_print_docs(compressed_docs)

#### 加载模型

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
model = HuggingFacePipeline.from_model_id(model_id=model_path,
        task="text-generation",
        device=0,
        pipeline_kwargs={
            "max_new_tokens": 4096, 
            "do_sample": True,
            "temperature": 0.2,
            "top_k": 1,
            "top_p": 0.9,
            "repetition_penalty": 1.1},
        model_kwargs={
            "torch_dtype": "auto",
            "low_cpu_mem_usage": True}
        )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

##### 定义目标模板

In [ ]:
qa_chain_refine = RetrievalQA.from_chain_type(
    model,
    retriever=db.as_retriever(search_kwargs={"k": 20}),
    chain_type="refine"
)
 
result = qa_chain_refine({"query": refine_query})
result["result"]


### 整理回答

#### 数据解析器

In [ ]:
que =result["result"]

initial_qa_template = (
    "[INST] <<SYS>>\n"
    "You are a helpful assistant. 你是一个乐于助人的助手。\n"
    "<</SYS>>\n\n"
    "以下为背景知识：\n"
    "{context_str}"
    "\n"
    "{format_instructions}"
    "\n"
    "请根据以上背景知识, 回答这个问题：{question}。"
    " [/INST]"
)


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template=initial_qa_template,
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions,"context_str":que}
)
_input = prompt.format_prompt(question=summary_query)
output = model(_input.to_string())
print(output)
output_parser.parse(output)
 

 以下是人工智能的突破性技术列表：

- 机器学习（Machine Learning）：通过训练模型来识别模式和做出决策的能力。
- 自然语言处理（Natural Language Processing）：使计算机能够理解、分析和生成人类语言的技术。
- 深度学习（Deep Learning）：一种基于神经网络的学习方法，可以自动提取特征并将其用于分类或预测任务。
- 计算机视觉（Computer Vision）：让计算机能够感知和理解图像和视频的能力。
- 强化学习（Reinforcement Learning）：通过与环境互动来学习最优策略的方法。
- 语音识别（Speech Recognition）：将人类语音转换为可理解的形式的能力。
- 机器人学（Robotics）：研究如何设计、制造和控制智能机器的行为和动作。


OutputParserException: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)